In [1]:
import os
import pandas as pd
import numpy as np
from keras.models import Model
from ch10 import construct_seq2seq_model
from nlpia.loaders import get_data, DATA_PATH

/usr/local/miniconda3/envs/nlpiaenv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
num_samples = 10000
data_path = os.path.join(DATA_PATH, 'movie_dialog.txt')  # preprocessed CMU movie dialogue samples

In [3]:
try:
    import cPickle as pickle
except ImportError:
    import pickle

from io import open

with open("../data/characters_stats.pkl", "rb") as filehandler:
    input_characters, target_characters, input_token_index, target_token_index = pickle.load(filehandler)

with open("../data/encoder_decoder_stats.pkl", "rb") as filehandler:
    num_encoder_tokens, num_decoder_tokens, max_encoder_seq_length, max_decoder_seq_length = pickle.load(filehandler)

In [10]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
lines = open(data_path).read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 44
Number of unique output tokens: 46
Max sequence length for inputs: 100
Max sequence length for outputs: 102


In [11]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [12]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [13]:
# model = construct_seq2seq_model(num_encoder_tokens, num_decoder_tokens)
from keras.layers import Input, LSTM, Dense
batch_size = 64    # <1>
epochs = 100       # <2>
num_neurons = 256  # <3>

encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(num_neurons, return_state=True)
_, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(num_neurons, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [14]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Instructions for updating:
Use tf.cast instead.
2019-04-12 17:37:00,216 WARNING:     tensorflow:323:            new_func From /usr/local/miniconda3/envs/nlpiaenv/lib/python3.6/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
2019-04-12 17:37:00,306 WARNING:     tensorflow:323:            new_func From /usr/local/miniconda3/envs/nlpiaenv/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:102: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 23s 3ms/step - loss: 1.0204 - acc: 0.0650 - val_loss: 0.8718 - val_acc: 0.0891
E

8000/8000 [==============================] - 20s 3ms/step - loss: 0.3603 - acc: 0.2327 - val_loss: 0.5861 - val_acc: 0.1631
Epoch 52/100
8000/8000 [==============================] - 20s 3ms/step - loss: 0.3573 - acc: 0.2337 - val_loss: 0.5895 - val_acc: 0.1634
Epoch 53/100
8000/8000 [==============================] - 20s 3ms/step - loss: 0.3739 - acc: 0.2282 - val_loss: 0.5874 - val_acc: 0.1631
Epoch 54/100
8000/8000 [==============================] - 20s 3ms/step - loss: 0.3587 - acc: 0.2327 - val_loss: 0.5908 - val_acc: 0.1620
Epoch 55/100
8000/8000 [==============================] - 20s 3ms/step - loss: 0.3526 - acc: 0.2349 - val_loss: 0.5951 - val_acc: 0.1628
Epoch 56/100
8000/8000 [==============================] - 20s 3ms/step - loss: 0.3485 - acc: 0.2364 - val_loss: 0.5982 - val_acc: 0.1628
Epoch 57/100
8000/8000 [==============================] - 20s 3ms/step - loss: 0.3446 - acc: 0.2377 - val_loss: 0.6023 - val_acc: 0.1617
Epoch 58/100
8000/8000 [==============================

In [23]:
model_path = os.path.join(DATA_PATH, 'ch10_train_seq2seq_keras')
model.save(model_path + '_model.h5')

/usr/local/miniconda3/envs/nlpiaenv/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_5 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_4/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_4/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [24]:
model_path

'/home/michael/python/projects/nlpia/src/nlpia/data/ch10_train_seq2seq_keras'

In [25]:
model.save_weights(model_path + '_weights.h5')

# Model Inference/Activation without training
Everything below can be run without rerunning training  
TODO: put these cells in a separate notebook named ch10_inference_...  

In [26]:
from keras.models import load_model

model_path = os.path.join(DATA_PATH, 'ch10_train_seq2seq_keras')
model = load_model(model_path + '_model.h5')

In [27]:
model.load_weights(model_path + '_weights.h5')

In [28]:
encoder_model = Model(encoder_inputs, encoder_states)
thought_input = [
    Input(shape=(num_neurons,)), Input(shape=(num_neurons,))]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=thought_input)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    inputs=[decoder_inputs] + thought_input,
    output=[decoder_outputs] + decoder_states)

/usr/local/miniconda3/envs/nlpiaenv/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  # This is added back by InteractiveShellApp.init_path()


In [29]:
>>> def decode_sequence(input_seq):
...     thought = encoder_model.predict(input_seq)  # <1>

...     target_seq = np.zeros((1, 1, output_vocab_size))  # <2>
...     target_seq[0, 0, target_token_index[stop_token]
...         ] = 1.  # <3>
...     stop_condition = False
...     generated_sequence = ''

...     while not stop_condition:
...         output_tokens, h, c = decoder_model.predict(
...             [target_seq] + thought) # <4>

...         generated_token_idx = np.argmax(output_tokens[0, -1, :])
...         generated_char = reverse_target_char_index[generated_token_idx]
...         generated_sequence += generated_char
...         if (generated_char == stop_token or
...                 len(generated_sequence) > max_decoder_seq_length
...                 ):  # <5>
...             stop_condition = True

...         target_seq = np.zeros((1, 1, output_vocab_size))  # <6>
...         target_seq[0, 0, generated_token_idx] = 1.
...         thought = [h, c]  # <7>

...     return generated_sequence